In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import vgg19
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from PIL import Image
import io


In [3]:
def load_and_process_img(image_path):
    max_dim = 512
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

# Replace these paths with your actual image paths
content_path = 'image1.png'
style_path = 'image2.png'

content_image = load_and_process_img(content_path)
style_image = load_and_process_img(style_path)


In [4]:
def get_model():
    vgg = vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    content_layers = ['block5_conv2']
    model_outputs = [vgg.get_layer(name).output for name in style_layers + content_layers]
    return Model([vgg.input], model_outputs)

model = get_model()


47104000/80134624 [================>.............] - ETA: 3s

In [ ]:
def get_content_loss(base_content, target):
    return tf.reduce_mean(tf.square(base_content - target))

def gram_matrix(input_tensor):
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)

def get_style_loss(base_style, gram_target):
    gram_style = gram_matrix(base_style)
    return tf.reduce_mean(tf.square(gram_style - gram_target))


In [ ]:
def get_feature_representations(model, content_path, style_path):
    content_image = load_and_process_img(content_path)
    style_image = load_and_process_img(style_path)
    style_outputs = model(style_image)
    content_outputs = model(content_image)
    style_features = [style_layer[0] for style_layer in style_outputs[:5]]
    content_features = [content_layer[0] for content_layer in content_outputs[5:]]
    return style_features, content_features

def compute_loss_and_grads(model, loss_weights, init_image, gram_style_features, content_features):
    with tf.GradientTape() as tape:
        outputs = model(init_image)
        style_output_features = outputs[:5]
        content_output_features = outputs[5:]

        style_score = 0
        content_score = 0

        weight_per_style_layer = 1.0 / float(len(style_layers))
        for target_style, comb_style in zip(gram_style_features, style_output_features):
            style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style)

        weight_per_content_layer = 1.0 / float(len(content_layers))
        for target_content, comb_content in zip(content_features, content_output_features):
            content_score += weight_per_content_layer * get_content_loss(comb_content[0], target_content)

        style_score *= loss_weights[0]
        content_score *= loss_weights[1]

        loss = style_score + content_score 
    return loss, tape.gradient(loss, init_image)


In [ ]:
def display_image(image):
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    plt.show()
